In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
%matplotlib inline

In [15]:
# Loading  data
df = pd.read_csv('UCI_Credit_Card_balanced.csv')

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()

In [18]:
scaler.fit(df.drop('Y',axis=1))

StandardScaler()

In [19]:
scaled_features = scaler.fit_transform(df.drop('Y',axis=1))

In [20]:
X = pd.DataFrame(scaled_features,columns=df.columns[:-1]).values
y = df['Y'].values

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate

In [22]:
def create_network():

    # create model
    model = Sequential()
        
    # hidden layer
    model.add(Dense(X.shape[1]/2, input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(BatchNormalization())

    # output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    # Return compiled network
    return model

In [23]:
# Wrap Keras model so it can be used by scikit-learn
ann = KerasClassifier(build_fn=create_network, epochs=200, verbose=0)

In [24]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=69)

In [25]:
t0 = datetime.now()
scores = cross_validate(ann, X, y, scoring=['accuracy','roc_auc','precision','recall','f1'], cv=cv, n_jobs=-1)
t1 = datetime.now()

In [13]:
print('Mean Accuracy: %.2f%% Std Dev: (+/- %.3f%%)' % (np.mean(scores['test_accuracy']*100), np.std(scores['test_accuracy']*100)))
print('Mean ROC AUC: %.2f%% Std Dev: (+/- %.3f%%)' % (np.mean(scores['test_roc_auc']*100), np.std(scores['test_roc_auc']*100)))
print('Mean Precision: %.2f%% Std Dev: (+/- %.3f%%)' % (np.mean(scores['test_precision']*100), np.std(scores['test_precision']*100)))
print('Mean Recall: %.2f%% Std Dev: (+/- %.3f%%)' % (np.mean(scores['test_recall']*100), np.std(scores['test_recall']*100)))
print('Mean F1 Score: %.2f%% Std Dev: (+/- %.3f%%)' % (np.mean(scores['test_f1']*100), np.std(scores['test_f1']*100)))
print("Execution Time: ", (t1 - t0))

Mean Accuracy: 76.58% Std Dev: (+/- 0.619%)
Mean ROC AUC: 83.76% Std Dev: (+/- 0.373%)
Mean Precision: 77.55% Std Dev: (+/- 1.300%)
Mean Recall: 71.21% Std Dev: (+/- 1.266%)
Mean F1 Score: 74.23% Std Dev: (+/- 0.622%)
Execution Time:  0:04:52.777684
